In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Google sheet - Get handle on it

In [2]:
!pip install --upgrade -q gspread

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")

# Download raw images

In [21]:
"""
Loop through rows of spreadsheet
Download images
"""
import requests
import os
row = 2
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
for row_counter in range(2,254):
    file_num = worksheet_pages.get_values("C"+str(row_counter))[0][0]
    image_url = worksheet_pages.get_values("D"+str(row_counter))[0][0]
    print(file_num, image_url)
    #ANT_FB_1932-003-0016.jpg
    image_file_name = "ANT_FB_" + file_num + ".jpg"
    img_data = requests.get(image_url).content
    image_file_path = os.path.join(raw_images_dir, image_file_name)
    with open(image_file_path, 'wb') as f:
        f.write(img_data)
        worksheet_pages.update_cell(row_counter, 8, image_file_name)

FB TowerCover https://pi.lib.uchicago.edu/1001/org/ochre/c39123df-8f09-4df8-9126-7f14610c8243&load


# Split images

In [314]:
import requests
import os
import cv2
from PIL import Image as im
# split pages
raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_images/'
raw_split_pages_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_split_pages/'

row = 2
for row_counter in range(2,254):
    file_name = worksheet_pages.get_values("H"+str(row_counter))[0][0]
    image_file_path = os.path.join(raw_images_dir,file_name)
    img = cv2.imread(image_file_path)
  
    # Shape of the image
    print(file_name, "Shape:", img.shape)
    img_shape = img.shape

    page1_file_name = file_name[:-4]+"_p1.jpg"
    page1_file_path = os.path.join(raw_split_pages_dir,
                                       page1_file_name)
    page2_file_name = file_name[:-4]+"_p2.jpg"
    page2_file_path = os.path.join(raw_split_pages_dir,
                                   page2_file_name)

    if img_shape[1]>800 or (img_shape[1] > img_shape[0]):
        num_pages = 2
        page1 = img[0:img_shape[0],0:int(img_shape[1]/2)]
        page2 = img[0:img_shape[0],int(img_shape[1]/2):int(img_shape[1])]
        cv2.imwrite(page1_file_path, page1)
        cv2.imwrite(page2_file_path, page2)
    else:
        num_pages = 1      
        cv2.imwrite(page1_file_path, img)
        page2_file_name = ""
        if os.path.exists(page2_file_path):
            print("removing redundant page 2 file", page2_file_path)           
            os.remove(page2_file_path)

    # update the sheet as a range to prevent Google sheets being overloaded with
    # too many writes
    print(page1_file_name,
      "",
      page2_file_name,
      num_pages,
      str(img.shape[0]),
      str(img.shape[1]))

    worksheet_pages.update('I' +
                       str(row_counter) +
                       ':O' +
                       str(row_counter),
                       [[page1_file_name,
                         "",
                         page2_file_name,
                        "",
                        num_pages,
                        str(img.shape[0]),
                        str(img.shape[1])]])     

ANT_FB_1932-002-0000.jpg Shape: (756, 1050, 3)
ANT_FB_1932-002-0000_p1.jpg  ANT_FB_1932-002-0000_p2.jpg 2 756 1050


In [ ]:
import sys
import cv2
from google.colab.patches import cv2_imshow
#@title Check image split { run: "auto", vertical-output: true }
sheet_row = 2 #@param {type:"integer"}
if sheet_row < 2 or sheet_row > 253:
   sys.exit()
num_pages_init = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
if num_pages_init == 1:
   print("This row has", num_pages_init, "page. Does it look correct? Should it be split?")
if num_pages_init == 2:  
   print("This row has", num_pages_init, "pages. Do they both look correct? Should they not be split?")
page1_file_name = worksheet_pages.get_values("I"+str(sheet_row))[0][0]
if len(page1_file_name.strip()) > 0:
    page1_file_path = os.path.join(raw_split_pages_dir,
                                   page1_file_name)
    page1 = cv2.imread(page1_file_path)
    print(page1_file_name, page1_file_path)
    cv2_imshow(page1)
try:    
    page2_file_name = worksheet_pages.get_values("K"+str(sheet_row))[0][0]
    if len(page2_file_name.strip()) > 0:
        page2_file_path = os.path.join(raw_split_pages_dir,
                                       page2_file_name)
        page2 = cv2.imread(page2_file_path)
        print(page2_file_name)
        cv2_imshow(page2)
except:
    print("no page 2")          

# Contrast images
Save a black and white version of each image

In [301]:
def convert_image_to_black_and_white(source_image_path, destination_image_path, threshold = 208):
    raw_img = cv2.imread(source_image_path)
    gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
    #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
    #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
    (thresh, binary) = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)
    cv2.imwrite(destination_image_path, binary)

In [ ]:
# convert gray images to black and white
import cv2 
from google.colab.patches import cv2_imshow

import os

raw_split_pages_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/raw_split_pages'
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/all_images/contrasted_split_pages' 
for raw_image_file_name in os.listdir(raw_split_pages_dir):
    if raw_image_file_name.endswith(".jpg"):
        print(raw_image_file_name)
        raw_img = cv2.imread(os.path.join(raw_split_pages_dir, raw_image_file_name))
        gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
        #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
        #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
        (thresh, binary) = cv2.threshold(gray_image, 208, 255, cv2.THRESH_BINARY)

        cv2.imwrite(os.path.join(contrast_images_dir,raw_image_file_name), binary)

In [ ]:
import sys
import cv2
from google.colab.patches import cv2_imshow
#@title Check image conversion to black and white { run: "auto", vertical-output: true }
sheet_row = 2 #@param {type:"integer"}
threshold = 205 #@param {type:"integer"}
if sheet_row < 2 or sheet_row > 253:
   sys.exit()
num_pages_init = int(worksheet_pages.get_values("M"+str(sheet_row))[0][0])
if num_pages_init == 1:
   print("This row has", num_pages_init, "page. Does it look correct? Should it be split?")
if num_pages_init == 2:  
   print("This row has", num_pages_init, "pages. Do they both look correct? Should they not be split?")
page1_file_name = worksheet_pages.get_values("I"+str(sheet_row))[0][0]
if len(page1_file_name.strip()) > 0:
    page1_file_path = os.path.join(raw_split_pages_dir,
                                   page1_file_name)
    page1bw_file_path = os.path.join(contrast_images_dir,
                                   page1_file_name)
    page1 = cv2.imread(page1_file_path)

    convert_image_to_black_and_white(page1_file_path, page1bw_file_path, threshold)
    page1bw = cv2.imread(page1bw_file_path)
    print(page1_file_name, page1_file_path, page1bw_file_path)
    cv2_imshow(page1bw)
    cv2_imshow(page1)

try:    
    page2_file_name = worksheet_pages.get_values("K"+str(sheet_row))[0][0]
    if len(page2_file_name.strip()) > 0:
        page2_file_path = os.path.join(raw_split_pages_dir,
                                       page2_file_name)
        page2bw_file_path = os.path.join(contrast_images_dir,
                                         page2_file_name)
        page2 = cv2.imread(page2_file_path)
        convert_image_to_black_and_white(page2_file_path, page2bw_file_path, threshold)        
        page2bw = cv2.imread(page2bw_file_path)
        print(page2_file_name, page2_file_path, page2bw_file_path)
        cv2_imshow(page2bw)
        cv2_imshow(page2)        
except:
    print("no page 2") 